In [41]:
import tensorflow as tf
import numpy as np



from datetime import datetime 
from coco_utils import load_coco_data, decode_captions
from image_utils import image_from_url, write_text_on_image
import numpy as np
import os
import sys
import argparse


    
class Config(object):
    def __init__(self):
        self.vocab_size =1004
        self.batch_size = 32
        self.initializer_scale =0.08
        self.H = 512 #hidden dimension
        self.T = 16 # caption length
        self.feature_len = 512
        self.W = 512 # embedding size
        self.num_epochs_per_decay = 8
        self.total_instances = 400135
        self.initial_learning_rate = 2.0
        
        
config = Config()


# minibatch data
def minibatch(data, batch_size=32, split='train'):
    size = data['%s_captions'%split].shape[0]
    mask = np.random.choice(size, batch_size)
    caption = data['%s_captions'%split][mask]
    image_idxs = data['%s_image_idxs'%split][mask]
    image_features = data['%s_features' % split][image_idxs]
    urls = data['%s_urls' % split][image_idxs]
    caption_in = caption[:,:-1]
    caption_out = caption[:,1:]
    mask = (caption_out != 0)
    return caption_in, caption_out, mask, image_features, urls

## wrong=.=
def _step_test(sess, data, batch_size, model, keep_prob):
    """
    Make a single gradient update for batch data. 
    """
    # Make a minibatch of training data
    caption_in, caption_out, _, image_features, urls = minibatch(data, split='val')

    
    # print out ground truth caption
    caption_in = caption_in[:, 0].reshape(-1, 1)
    
    state = None 
    final_preds = []
    current_pred = caption_in
    mask = np.zeros((batch_size, config.T))
    mask[:, 0] = 1
    
    # get initial state using image feature 
    feed_dict = {model['image_feature']: image_features}
    state = sess.run(model['initial_state'], feed_dict=feed_dict)
    
    # start to generate sentences
    for t in range(config.T):
        feed_dict={model['caption_in']: current_pred, 
                   model['initial_state']: state, 
                   model['input_mask']: mask}
            
        current_pred, state = sess.run([model['preds'], model['final_state']], feed_dict=feed_dict)
        
        current_pred = current_pred.reshape(-1, 1)
        
        final_preds.append(current_pred)
        
    return final_preds, urls


g = tf.Graph()
with g.as_default():
    # Build the model.
    model = buildModel(config, mode, inference_batch = BATCH_SIZE_INFERENCE)
    
    # run training 
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
    
        sess.run(init)
    
        model['saver'].restore(sess, directory + "savedSession/model0.ckpt")
          
        print("Model restured! Last step run: ", sess.run(model['global_step']))
        
        for i in range(TOTAL_INFERENCE_STEP):
            captions_pred, urls = _step_test(sess, data, BATCH_SIZE_INFERENCE, model, 1.0) # the output is size (32, 16)
            captions_pred = [unpack.reshape(-1, 1) for unpack in captions_pred]
            captions_pred = np.concatenate(captions_pred, 1)
            
            captions_deco = decode_captions(captions_pred, data['idx_to_word'])
            
            for j in range(len(captions_deco)):
                img_name = directory + 'image_' + str(j) + '.jpg'
                img = image_from_url(urls[j])
                write_text_on_image(img, img_name, captions_deco[j])


NameError: name 'mode' is not defined

In [53]:



def buildModel(config, mode):
    
    caption_in = tf.placeholder(tf.int32, [None, None], name='caption_in') 
    caption_out = tf.placeholder(tf.int32, [None, None], name='caption_out')
    caption_mask = tf.placeholder(tf.int32, [None, None], name='caption_mask')
    image_feature = tf.placeholder(tf.float32, [None, config.feature_len], name='image_feature')
    initializer = tf.random_uniform_initializer(
            minval = -0.08, maxval = 0.08)
    
    word_embedding = None 
    total_loss = None
    # N.T
    target_loss = None
    
    global_step = tf.Variable(
    initial_value=0,
    name="global_step",
    trainable=False,
    collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
    
    # word embedding
    with tf.variable_scope("word_embedding"):
        embedding_map = tf.get_variable(
            name ="map",
            shape = [config.vocab_size, config.W],
            initializer = initializer)
        #captions_in is N.T
        word_embedding = tf.nn.embedding_lookup(embedding_map, caption_in)

    # image feature embedding
    with tf.variable_scope('feature_embedding'):
        feature_embedding = tf.contrib.layers.fully_connected(
          inputs=image_feature,
          num_outputs= config.W,
          activation_fn=None,
          weights_initializer= initializer,
          biases_initializer=None)
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(
            num_units = config.H, state_is_tuple =True)
    
    #drop out?
    
    with tf.variable_scope('lstm', initializer=initializer) as lstm_scope:
        zero_state = lstm_cell.zero_state(
            batch_size=config.batch_size, dtype=tf.float32)
        _, initial_state = lstm_cell(feature_embedding, zero_state)

        lstm_scope.reuse_variables()

        caption_len = tf.reduce_sum(caption_mask,1)
        lstm_out, final_state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                                  inputs = word_embedding,
                                                  sequence_length = caption_len,
                                                  initial_state = initial_state,
                                                  dtype = tf.float32,
                                                  scope = lstm_scope)
    
    print('shape', lstm_out.get_shape())
    lstm_out = tf.reshape(lstm_out, [-1, lstm_cell.output_size])
    print('after shape', lstm_out.get_shape())
    print('ouput_size',lstm_cell.output_size )
    
    with tf.variable_scope('logits'):
        w = tf.get_variable('w', [lstm_cell.output_size, config.vocab_size], initializer=initializer)
        b = tf.get_variable('b', [config.vocab_size], initializer=tf.constant_initializer(0.0))
        # N.H * H* v =N*V, bias is zero
        logits = tf.matmul(lstm_out,w)+b
    
    output = tf.reshape(caption_in, [-1])
    mask = tf.to_float(tf.reshape(caption_mask,[-1]))
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels= output,
                                                         logits = logits)
    
    batch_loss = tf.div(tf.reduce_sum(tf.multiply(loss, mask)),
                        tf.reduce_sum(mask),
                        name="batch_loss")
    tf.losses.add_loss(batch_loss)
    total_loss = tf.losses.get_total_loss()
    
    # predictions
    softmax = tf.nn.softmax(logits)
    preds = tf.argmax(softmax, 1)

    return dict(
        total_loss = total_loss, 
        global_step = global_step,
        image_feature = image_feature, 
        caption_mask = caption_mask, 
        caption_in = caption_in, 
        caption_out = caption_out,  
        final_state = final_state,
        initial_state = initial_state, 
        softmax = softmax,
        preds = preds, 
        saver = tf.train.Saver()
    )
    
def validation(sess, inputs, features, model):
    # get the first word
    inputs = inputs[:,0].reshape(-1,1)
    # initial state
    feed_dict = {model['image_feature']: features}
    state = sess.run(model['initial_state'], feed_dict=feed_dict)
    preds =[]
    mask = np.zeros((batch_size, config.T))
    mask[:, 0] = 1
    
    for i in range(config.T):
        feed_dict={model['caption_in']: inputs,
                  model['initial_state']:state,
                  model['caption_mask']: mask}
        current_pred, state = sess.run([model['preds'], model['final_state']], feed_dict=feed_dict)
        current_pred = current_pred.reshape(-1, 1)
        preds.append(current_pred)
    return preds, urls

def main(_):
    data = load_coco_data()
    g = tf.Graph()
    with g.as_default():
        model = buildModel(config, mode='train')
        
        # decay learning rate
        num_batches = config.total_instances/config.batch_size
        decay_steps = int(num_batches*config.num_epochs_per_decay)
        learning_rate = config.initial_learning_rate
        
        
        def _decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(learning_rate,
                                             global_step,
                                             decay_steps = decay_steps,
                                             decay_rate=0.5,
                                             staircase=True)
        
        learning_rate_decay_fn = _decay_fn
        train_op = tf.contrib.layers.optimize_loss(loss=model['total_loss'],
                                                  global_step = model['global_step'],
                                                  learning_rate = learning_rate,
                                                  optimizer = 'SGD',
                                                  clip_gradients = None,
                                                  learning_rate_decay_fn =learning_rate_decay_fn)
        
        init = tf.global_variables_initializer()
        
        with tf.Session() as sess:
            sess.run(init)

            num_epochs = 10000
            total_runs = int((config.total_instances/config.batch_size)*num_epochs)
            
            loss_history = []
            saved_dir ='/Users/amber/Desktop/savedSession/'
            sample_dir = '/Users/amber/Desktop/sample_caption/'
            print("\n\nTotal training iter: ", total_runs, "\n\n")
            time_now = datetime.now()
            for t in range(total_runs):            
                caption_in, caption_out, mask, image_features, urls = minibatch(data)
                _, total_loss_value= sess.run([train_op, model['total_loss']], 
                                  feed_dict={model['image_feature']: image_features, 
                                             model['caption_in']: caption_in, 
                                             model['caption_out']: caption_out, 
                                             model['caption_mask']: mask})
                loss_history.append(total_loss_value)
                
                #validation
                if (t+1)%10 == 0:
                    if not os.path.exists(sample_dir):
                        os.makedirs(sample_dir)
                    captions_pred, urls = validation(sess, caption_in, image_features, model)
                    captions_pred = [unpack.reshape(-1, 1) for unpack in captions_pred]
                    captions_pred = np.concatenate(captions_pred, 1)
                    
                    captions_deco = decode_captions(captions_pred, data['idx_to_word'])
                    
                    for j in range(len(captions_deco)):
                        img_name = os.path.join(sample_dir, 'image_{}.jpg'.format(j))
                        img = image_from_url(urls[j])
                        write_text_on_image(img, img_name, captions_deco[j])
                
                # print accuracy
                if(t+1)%50 == 0:
                    print('(Iteration %d / %d) loss: %f, and time eclipsed: %.2f minutes' % (
                        t + 1, num_iterations, float(loss_history[-1]), (datetime.now() - time_now).seconds/60.0))
                #save model
                if(t+1)%5000 == 0:
                    if not os.path.exists(saved_dir):
                        os.makedirs(saved_dir)
                    checkpoint_name = '_checkpoint{}.ckpt'.format(t+1)
                    save_path = model['saver'].save(sess, os.path.join(saved_dir, checkpoint_name))
                    
main(_)

            
    
        


shape (32, ?, 512)
after shape (?, 512)
ouput_size 512


Total training iter:  125042187 




InvalidArgumentError: assertion failed: [Expected shape for Tensor lstm/sequence_length:0 is ] [32] [ but saw shape: ] [10]
	 [[Node: lstm/lstm/Assert/Assert = Assert[T=[DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/cpu:0"](lstm/lstm/All, lstm/lstm/Assert/Assert/data_0, lstm/lstm/stack, lstm/lstm/Assert/Assert/data_2, lstm/lstm/Shape_1)]]

Caused by op 'lstm/lstm/Assert/Assert', defined at:
  File "/Users/amber/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/amber/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/amber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/amber/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/amber/anaconda/lib/python3.6/site-packages/ipykernel